## 將所有連結存成文字檔

## 寫進raw data

## 寫進乾淨的資料

In [1]:
import requests
from bs4 import BeautifulSoup
import math
import re
from datetime import date,datetime, timedelta 
import time
import traceback
import string  


#################for db
def connect(dbname,password,host,user,port=3306):
    import MySQLdb
    # 承德電腦 host="10.120.30.1"
    # localhost host = '127.0.0.1'
    db = MySQLdb.connect(host=host,user="root",passwd=password,db=dbname,charset="utf8")
    return db
def insert_news(db,category=None,title=None,newsDate=None,journalist=None,content=None,hit=None,url=None,source="ETToday"):
    cur = db.cursor()
    cur.execute("insert into news(class,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,url,source])
    #cur.execute("insert into news(class,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,url,source])
    db.commit()
def insert_comments(db,commentDate,commentTime,commenter,commenterUrl,comment,url,remark=None):
    cur = db.cursor()
    cur.execute("insert into comments(commentDate,commentTime,commenter,commenterUrl,comment,url)values(%s,%s,%s,%s,%s,%s)" ,[commentDate,commentTime,commenter,commenterUrl,comment,url])
    db.commit()
def insert_newsRaw(db,url,newsRaw):
    cur = db.cursor()
    cur.execute("insert into newsRaw(`class`,url,newsRaw,source)values(%s,%s,%s,%s)",[category,url,newsRaw,source])
    db.commit()
def get_newsRaw(db,url):
    cur = db.cursor()
    cur.execute("select newsRaw from newsRaw where url = %s",[url])
    return cur.fetchone()
def close(db):
    db.close()
#################for db


#################Exception Handler
def EH(breakpoint=None,aLink=None):            
    f=open("Ubn.log","a")
    ExList=[
        "Log at: " + str(datetime.today()) + "\n",
        "\t -News url: " + str(aLink) + "\n",
        "\t -BreakPoint: "+ str(breakpoint) + "\n",
        "\t -Exception Traceback: \n", 
        traceback.format_exc() + "\n"
    ]
    f.writelines(ExList)
    f.close
#################Exception Handler


#################parse_page_and_insert_into_db
hit = None
sentinel = object()
def parse_page(db="news",passwd="",host="localhost",user="root",rawTable="newsRaw2"):
    import MySQLdb.cursors
    conn = MySQLdb.connect(user=user,passwd=passwd,db=db,cursorclass = MySQLdb.cursors.SSCursor,charset="utf8")
    #conn = MySQLdb.connect(user=user,passwd=passwd,db=db)
    cur = conn.cursor()
    query="select newsRaw,url,`class` from %s;"%rawTable
    cur.execute(query)
    raw = cur.fetchone()
    while raw is not None:
        try:
            contRaw=raw[0].strip()
            #print type(contRaw),contRaw
            soup=BeautifulSoup(contRaw)
            #print type(soup),soup
            if "'" in aLink:
                aLink=re.sub("'","",raw[1])
            else:
                aLink=raw[1]
            #print soup.select('.content')
            contentre=re.search(u"(.*)【(\d{4}-\d{2}-\d{2})\/(.*)\/.*\/(.*)】",soup.select(".story")[0].text)
            source=contentre.group(3)
            #########category breakpoint
            try:
                category=contentre.group(4)
            except:
                EH(breakpoint="category",aLink=aLink)
            #########category breakpoint

            #########title breakpoint
            try:
                title=soup.select(".story_title")[1].text.strip()
            except:
                EH(breakpoint="title",aLink=aLink)
            #########title breakpoint

            ########Date&Time breakpoint
            try:
                newsDate=contentre.group(2)
                newsTime=None
            #newsTime=timeRegex.group(3)
            except:
                EH(breakpoint="Date&Time",aLink=aLink)
            ########Date&Time breakpoint

            #########content breakpoint
            try:
                content=re.sub(u"【.*】","",soup.select(".story")[0].text).strip()
            except:
                EH(breakpoint="content",aLink=aLink)
            #########content breakpoint        

            #########journist breakpoint
            jor = soup.select(".story_author")[0].text
            try:
                journalist=re.search(u'記者([^─】╱/]+)', jor).group(1).strip()
            except:
                try:
                    journalist=re.search(u'【?(.*)(】|[^】] )', jor).group(1).strip()
                except:
                    EH(breakpoint='journalist',aLink=aLink) 
                    print "jor: ",aLink," WTF"
            #########journist breakpoint

            '''#########comment breakpoint
            comments=[]
            try:
                commLink0 = soupCont.find("div",re.compile('fb-comments.*'))
                commLink1= commLink0.select(' span > iframe')[0]['src']
                commLink=re.sub(r"numposts=\d{1,3}&", "numposts=100&", commLink1)
                resComm = requests.get(commLink)
                resComm.encoding = "utf-8"
                soupComm = BeautifulSoup(resComm.text)      
                commentBlock=soupComm.select('.postContainer.fsl.fwb.fcb')
                for comm in commentBlock: 
                    dt=re.search("(\d\d\d\d).(\d{1,2}).(\d{1,2}). (\d{1,2}\:\d{1,2})",comm.select('abbr')[0]['title'])
                    commentDate="%s-%02d-%02d"%(dt.group(1),int(dt.group(2)),int(dt.group(3)))
                    commentTime=dt.group(4)
                    commenter=comm.select('.profileName')[0].text
                    commenterUrl=comm.select('.profileName')[0]['href']
                    comment=comm.select('.postText')[0].text
                    comments.append({'commentDate':commentDate,'commentTime':commentTime,'commenter':commenter,'commenterUrl':commenterUrl,'comment':comment,'url':aLink,'remark':None})
            except Exception as e:
                EH('comment',newsDate=None,category=None,aLink=None) 
            #########comment breakpoint'''

            #################connect to db
            #print category,title,newsDate,newsTime,journalist,content,hit,aLink,commentDate,commentTime,commenter,comment
            try:
                #print "result:",category,title,newsDate,newsTime,journalist,content,hit,aLink
                conn2 = MySQLdb.connect(db=db,passwd=passwd,host=host,user=user,charset="utf8")
                cur2 = conn2.cursor()
                #cur2.execute("insert into news(`class`,title,newsDate,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,journalist,content,hit,aLink,"Apple Daily"])
                cur2.execute("insert into news(`class`,title,newsDate,newsTime,journalist,content,hit,url,source) values(%s,%s,%s,%s,%s,%s,%s,%s,%s)",[category,title,newsDate,newsTime,journalist,content,hit,aLink,source])
                conn2.commit()
                #insert_news(conn,category,title,newsDate,newsTime,jornalist,content,hit,aLink)
                print "complete inserting: ",aLink,"\n"
            except:
                print "something not write in DB \n", traceback.format_exc(),aLink,"\n"
            finally:
                cur2.close()
                conn2.close()
        except:
            print "fail loop",aLink,"\n"
            print traceback.format_exc()
        #################connect to db
        raw = cur.fetchone()
    cur.close()
    conn.close()
    print "all finished!"
#################parse_page_and_insert_into_db

In [ ]:
parse_page(db="ecodaily2",passwd="oblivn0374",host="localhost",user="root",rawTable="newsRaw")